# Streaming
## 流计算概述

### 静态数据和流数据

**流数据的特征**
- 数据快速持续到达，潜在大小也许是无穷无尽
- 数据来源众多，格式复杂
- 数据量大，但是不十分关注存储，一旦经过处理，要么被丢弃，要么被归档存储
- 注重数据的整体价值，不过分关注个别数据
- 数据顺序颠倒，或者不完整，系统无法控制将要处理的新到达的数据元素的顺序

#### 批量计算和实时计算
- 批量计算：充裕时间处理静态数据，如Hadoop
- 流数据不适合采用批量计算，因为流数据不适合用传统的关系模型建模
- 流数据必须采用实时计算，响应时间为秒级

#### 流计算概念
- 流计算：实时获取来自不同数据源的海量数据，经过实时分析处理，获得有价值的信息
- 流计算的理念：**数据的价值随着时间的流逝而降低**


**流计算系统的需求**
- 高性能：处理大数据的基本要求，如每秒处理几十万条数据
- 海量式：支持TB级甚至PB级的数据规模
- 实时性：保证较低的延迟时间，达到秒级别，甚至是毫秒级别
- 分布式：支持大数据的基本架构，必须能够平滑扩展
- 易用性：能够快速进行开发和部署
- 可靠性：能可靠地处理流数据


#### 流计算框架
- 三大常见的流计算框架和平台：商业级的流计算平台、开源流计算框架、公司为支持自身业务开发的流计算框架
- 商业级：IBM InfoSphere Stream和IBM StreamBase
- 较为常见的开源流计算框架：
    - Twitter Storm：免费、开源的分布式实时计算系统，可简单、高效、可靠地处理大量的流数据
    - Yahoo!S4 (Simple Scalable Streaming System):开源流计算平台，是通用的、分布式的、可扩展的、分区容错的、可插拔的流式系统
    
- 公司为自身业务开发的流计算框架
    - Facebook Puma
    - Dstream
    - 银河流数据处理平台
    
#### 流计算处理流程
1.概述
- 传统的数据处理流程，需要先采集数据并存储在关系数据库等数据管理系统中，之后由用户通过查询操作和数据管理系统进行交互
$$
    用户查询\rightarrow数据管理系统\rightarrow查询结果
$$

- 传统的数据处理流程隐含了两个前提：
    - **存储的数据是旧的。**存储的静态数据是过去某一时刻的快照，这些数据在查询时可能已经不具备时效性
    - 需要用户主动发出查询来获取结果  
- 流计算的处理流程一般包含三个阶段：数据实时采集、数据实时计算、实时查询服务

2.数据实时采集
- 数据实时采集阶段通常采集多个数据源的海量数据，需要保证实时性、低延迟与稳定可靠
- 以日志数据为例，由于分布式集群的广泛应用，数据分散在不同的机器上，因此需要实时汇总来自不同机器上的日志数据
- 目前有许多互联网公司发布的开源分布式日志采集系统均可满足每秒数百MB的数据采集和传输需求，如：
    - Facebook的Scribe
    - LinkedIn的Kafka
    - 淘宝的Time Tunnel
    - 基于Hadoop的Chukwa和Flume
    
    
3.数据实时计算
- 数据实时计算阶段对采集的数据进行实时的分析和计算，并反馈实时结果
- 经流处理系统处理后的数据，可视情况进行存储，以便之后再进行分析计算。在时效性要求较高的场景中，处理之后的数据也可以直接丢弃


4.实时查询服务
- 实施查询服务：经由流计算框架得出的结果可供用户进行实时查询、展示或存储
- 传统的数据处理流程，用户需要主动发出查询才能获得想要的结果。而流处理流程中，实时查询服务可以不断更新结果，并将用户所需的结果实时推送给用户 
- 虽然通过对传统的数据处理系统进行**定时**查询，也可以实现不断地更新结果和结果推送，但始终有着本质区别



**流处理系统与传统的数据处理系统的不同**
- 流处理系统处理的是实时的数据，而传统的数据处理系统处理的是预先存储好的静态数据
- 用户通过流处理系统获取的是实时结果，而通过传统的数据处理系统，获取的是过去某一时刻的结果
- 流处理系统无需用户主动发出查询，实时查询服务可以主动将实时结果推送给用户


   

## Spark Streaming
### Spark Streaming设计
- Spark Streaming可整合多种输入数据源，如Kafka、Flume、HDFS，甚至是普通的TCP套接字。经过处理后的数据可存储至文件系统、数据库，或显示在仪表盘里
- Spark Streaming的基本原理是将实时输入数据流以时间片（秒级）为单位进行拆分，然后经Spark引擎以类似批处理的方式处理每个时间片数据
- Spark Streaming最主要的抽象是DStream(Discretized Stream, 离散化数据流)，表示连续不断的数据流。在内部实现上，Spark Streaming的输入数据按时间片（如1秒）分成一段一段，每一段数据转换为Spark中的RDD，这些分段就是DStream，并且对DStream的操作都最终变为相对应的RDD操作


### Spark  Streaming与Storm的对比
- Spark Streaming和Storm最大的区别在于，Spark Streaming无法实现毫秒级的流计算，而Storm可以实现毫秒级响应
- Spark Streaming构建在Spark上，一方面是因为Spark的低延迟执行引擎可以用于实时计算，另一方面，相比于Storm，RDD数据集更容易做高效的容错处理
- Spark Streaming采用的小批量处理的方式使得它可以同时兼容批量和实时数据处理的逻辑和算法





## DStream操作概述
### Spark Streaming工作机制
- 在Spark Streaming中，会有一个组件Receiver，作为一个长期运行的task跑在一个Executor上
- 每个Receiver都会负责一个input DStream（比如从文件中读取数据的文件流，比如套接字流，或者从Kafka中读取的一个输入流等等）
- Spark Streaming通过input DStream与外部数据源进行连接，读取相关数据

### Spark Streaming程序的基本步骤
1.通过创建输入DStream来定义输入源  
2.通过对DStream应用转换操作和输出操作来定义流计算  
3.用streamingContext.start()来开始接收数据和处理流程  
4.通过streamingContext.awaitTermination()方法来等待处理结束（手动结束或因为错误而结束）  
5.可以通过streamingContxt.stop()来手动结束流计算  


In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext
conf = SparkConf().setAppName('TestDStream').setMaster('local[2]')
sc = SparkContext(conf=conf)
ssc = StreamingContext(sc, 1)

## 基本输入源
### 文件流
1.在pyspark中创建文件流
```shell
$ cd /opt/software/spark/data/streaming
$ mkdir logfile
$ cd logfile

```

In [ ]:
#创建文件流
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext

conf = SparkConf().setAppName('fileStream').setMaster('local')
sc = SparkContext(conf=conf)
ssc = StreamingContext(sc, 10)
lines = ssc.textFileStream('file:///opt/software/spark/data/streaming/AFINN-111.txt')
words = lines.flatMap(lambda line: line.split(''))
wordCounts = words.map(lambda x: (x, 1)).reduceByKey(lambda a,b: a+b)
wordCounts.pprint()
ssc.start()
ssc.awaitTermination()